In [ ]:
import numpy as np
import pandas as pd
import ibm_db
import ibm_db_dbi
import matplotlib.pyplot as plt
from matplotlib import style
from scipy.stats import zscore
style.use('fivethirtyeight')
%matplotlib inline
pd.set_option('display.max_colwidth',-1)

In [ ]:
#values for database connection
dsn_driver = "IBM DB2 JDBC DRIVER"
dsn_database = "BLUDB"            
dsn_hostname = "" # taken out for privacy
dsn_port = "50000"                
dsn_protocol = "TCPIP"          
dsn_uid = ""        # taken out for privacy
dsn_pwd = ""       # taken out for privacy
dsn_schema = ""   # taken out for privacy

In [ ]:
dsn = (
    "DRIVER={{IBM DB2 JDBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "UID={3};"
    "PWD={4};"
    "CurrentSchema={5}").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd, dsn_schema)

conn = ibm_db.connect(dsn, "", "")
conn_wrap = ibm_db_dbi.Connection(conn)

# Read excel file for earn clean data

In [ ]:
df = pd.read_excel('clean_df.xlsx',sheet_name='Sheet1')

In [ ]:
df.columns

In [ ]:
# filter out any negative sales amount
df = df[df[' '] > 0] 

In [ ]:
# sanity check
df[df[' '] < 0]

In [ ]:
# filter out test ID for test tenant
df = df[df[' '] != ' ']

In [ ]:
# sanity check again
df[df[' '] == ' ']

# total spending per month per account

In [ ]:
# group by account number and month only in 2018
df1 = df[df[' '] == 2018].groupby([' ',' '])[' '].sum().reset_index(name='TOTALSPENDPERMONTH')

In [ ]:
display(df1)

## totalSpendPerMonth normalize

In [ ]:
# group by account number
df1mean = df1.groupby([' '])['TOTALSPENDPERMONTH'].mean().reset_index(name='AVGTOTALPERYEAR')

In [ ]:
display(df1mean)

In [ ]:
result1 = df1.merge(df1mean)
result1['TOTALSPENDNORM']= (result1['TOTALSPENDPERMONTH'] - result1['AVGTOTALPERYEAR']) / result1['AVGTOTALPERYEAR']

In [ ]:
display(result1)

## Z score

In [ ]:
def z_score(s):
    mean = s.mean()
    std = s.std()
    return s.apply(lambda x: (x - mean) / std)

In [ ]:
result1['SPENDZ'] = z_score(result1['TOTALSPENDNORM'])

In [ ]:
result1[result1['SPENDZ'] > 4].sort_values(['SPENDZ'], ascending=False)

In [ ]:
result1[result1['SPENDZ'] < -4].sort_values(['SPENDZ'], ascending=True)

# avg spending per month per account

In [ ]:
# group by account number, month and only in 2018
df2 = df[df[' '] == 2018].groupby(['',''])[''].mean().reset_index(name='AVGSPENDPERMONTH')

In [ ]:
display(df2)

## avg spending per account per month normalize

In [ ]:
# group by account number 
df2mean = df2.groupby([' '])['AVGSPENDPERMONTH'].mean().reset_index(name='AVGSPENDPERYEAR')

In [ ]:
display(df2mean)

In [ ]:
result2 = df2.merge(df2mean)
result2['AVGSPENDNORM']= (result2['AVGSPENDPERMONTH'] - result2['AVGSPENDPERYEAR']) / result2['AVGSPENDPERYEAR']

In [ ]:
display(result2)

## Zscore 

In [ ]:
result2['AVGSPENDZ'] = z_score(result2['AVGSPENDNORM'])

In [ ]:
result2[result2['AVGSPENDZ'] > 4].sort_values(['AVGSPENDZ'], ascending=False)

In [ ]:
result2[result2['AVGSPENDZ'] < -4].sort_values(['AVGSPENDZ'], ascending=True)

# Number of transaction per month per account

In [ ]:
# group by account number and month in 2018
# count number of transaction 
df3 = df[df[''] == 2018].groupby(['',''])[''].size().reset_index(name='TRANSNUM')

In [ ]:
display(df3)

## Number of transac normalize

In [ ]:
# group by account number
df3mean = df3.groupby([''])['TRANSNUM'].mean().reset_index(name='AVGTRANSNUMPERYEAR')

In [ ]:
display(df3mean)

In [ ]:
result3 = df3.merge(df3mean)
result3['TRANSNUMNORM']= (result3['TRANSNUM'] - result3['AVGTRANSNUMPERYEAR']) / result3['AVGTRANSNUMPERYEAR']

In [ ]:
display(result3)

## Z score

In [ ]:
result3['TRANSNUMZ'] = z_score(result3['TRANSNUMNORM'])

In [ ]:
result3[result3['TRANSNUMZ'] > 4].sort_values(['TRANSNUMZ'], ascending=False)

In [ ]:
result3[result3['TRANSNUMZ'] < -4]

# Max spend per month per account

In [ ]:
# group by account number and month in 2018
# max of spending 
df4 = df[df[''] == 2018].groupby(['',''])[''].max().reset_index(name='MAXSPENDPERMONTH')

In [ ]:
display(df4)

## Max spend per month normalize

In [ ]:
# group by account number and month 
df4mean = df4.groupby([''])['MAXSPENDPERMONTH'].mean().reset_index(name='AVGMAXSPENDPERYEAR')

In [ ]:
display(df4mean)

In [ ]:
result4 = df4.merge(df4mean)
result4['MAXSPENDNORM']= (result4['MAXSPENDPERMONTH'] - result4['AVGMAXSPENDPERYEAR']) / result4['AVGMAXSPENDPERYEAR']
display(result4)

## Z score

In [ ]:
result4['MAXSPENDZ'] = z_score(result4['MAXSPENDNORM'])

In [ ]:
result4[result4['MAXSPENDZ'] > 4]

In [ ]:
result4[result4['MAXSPENDZ'] < -4]

# Max earn point per month per account 

In [ ]:
# group by account number, month in 2018
# get max of earn point 
df5 = df[df[''] == 2018].groupby(['',''])[''].max().reset_index(name='MAXEARNPERMONTH')

In [ ]:
display(df5)

## Max earn point normalize

In [ ]:
# group by account number
df5mean = df5.groupby([''])['MAXEARNPERMONTH'].mean().reset_index(name='AVGMAXEARNPERYEAR')

In [ ]:
display(df5mean)

In [ ]:
result5 = df5.merge(df5mean)
result5['MAXEARNNORM']= (result5['MAXEARNPERMONTH'] - result5['AVGMAXEARNPERYEAR']) / result5['AVGMAXEARNPERYEAR']

In [ ]:
display(result5)

## Z score

In [ ]:
result5['MAXEARNZ'] = z_score(result5['MAXEARNNORM'])

In [ ]:
result5[result5['MAXEARNZ'] > 4].sort_values(['MAXEARNZ'], ascending=False)

In [ ]:
result5[result5['MAXEARNZ'] < -4].sort_values(['MAXEARNZ'], ascending=True)

# Average earn point per month per account

In [ ]:
# group by account number and month in 2018
# mean aggregation of point earned
df6 = df[df[''] == 2018].groupby(['',''])[''].mean().reset_index(name='AVGEARNPOINT')

In [ ]:
display(df6)

## Avg earn normalize

In [ ]:
# group by account number
df6mean = df6.groupby([''])['AVGEARNPOINT'].mean().reset_index(name='AVGEARNPERYEAR')

In [ ]:
display(df6mean)

In [ ]:
result6 = df6.merge(df6mean)
result6['AVGEARNNORM']= (result6['AVGEARNPOINT'] - result6['AVGEARNPERYEAR']) / result6['AVGEARNPERYEAR']

In [ ]:
display(result6)

## Z score

In [ ]:
result6['AVGEARNZ'] = z_score(result6['AVGEARNNORM'])

In [ ]:
result6[result6['AVGEARNZ'] > 4].sort_values(['AVGEARNZ'], ascending=False)

In [ ]:
result6[result6['AVGEARNZ'] < -4].sort_values(['AVGEARNZ'], ascending=True)

# Max burn point per month per account

In [ ]:
# Max burn point per month per account
sql10 = """ WITH MPxMPD (MBID, BURNPOINT, BURNMONTH) AS
            ( taken out for privacy reason
            )
             
           , BURNxPROFILE(ACCOUNTNUM, MAXBURNPOINT, MONTH) AS
            ( taken out for privacy reason
             )
        SELECT *
        FROM BURNxPROFILE
        """

df7 = pd.read_sql(sql10, conn_wrap)

## max burn point normalize

In [ ]:
# group by account number
df7mean = df7.groupby([''])['MAXBURNPOINT'].mean().reset_index(name='AVGMAXBURNPERYEAR')
#display(df7mean)

In [ ]:
result7 = df7.merge(df7mean)
result7['MAXBURNNORM']= (result7['MAXBURNPOINT'] - result7['AVGMAXBURNPERYEAR']) / result7['AVGMAXBURNPERYEAR']
#display(result7)

## Z score

In [ ]:
result7['MAXBURNZ'] = z_score(result7['MAXBURNNORM'])

In [ ]:
result7[result7['MAXBURNZ'] > 4].sort_values(['MAXBURNZ'], ascending=False)

In [ ]:
result7[result7['MAXBURNZ'] < -4]

# Average burn point per month per account

In [ ]:
sql11 = """
        WITH MPxMPD (MBID, BURNPOINT, BURNMONTH) AS
            ( ...... taken out for privacy reason ....
            )
        
           , BURNxPROFILE(ACCOUNTNUM, AVGBURNPOINT, MONTH) AS
            ( ...... taken out for privacy reason ....
            )
             
        SELECT *
        FROM BURNxPROFILE
        
        
        """

df8 = pd.read_sql(sql11, conn_wrap)

## avg burn point normalize

In [ ]:
# group by account number
df8mean = df8.groupby([''])['AVGBURNPOINT'].mean().reset_index(name='AVGBURNPERYEAR')
#display(df8mean)

In [ ]:
result8 = df8.merge(df8mean)
result8['AVGBURNNORM']= (result8['AVGBURNPOINT'] - result8['AVGBURNPERYEAR']) / result8['AVGBURNPERYEAR']
#display(result8)

## Z score

In [ ]:
result8['AVGBURNZ'] = z_score(result8['AVGBURNNORM'])

In [ ]:
result8stdmore = result8[result8['AVGBURNZ'] > 4]

In [ ]:
result8stdmore

In [ ]:
result8stdmore.sort_values(['AVGBURNZ'], ascending=False)

In [ ]:
result8[result8['AVGBURNZ'] < -4]

# Combine data frames

## sort dataframes

In [ ]:
result1 = result1.sort_index(axis=0)
result2 = result2.sort_index(axis=0)
result3 = result3.sort_index(axis=0)
result4 = result4.sort_index(axis=0)
result5 = result5.sort_index(axis=0)
result6 = result6.sort_index(axis=0)
result7 = result7.sort_index(axis=0)
result8 = result8.sort_index(axis=0)

## Join frames

In [ ]:
# first 2 columns is account number and month
frames = [result1.loc[:,['','','TOTALSPENDNORM']], 
          result2.loc[:,['','','AVGSPENDNORM']], 
          result3.loc[:,['','','TRANSNUMNORM']], 
          result4.loc[:,['','','MAXSPENDNORM']], 
          result5.loc[:,['','','MAXEARNNORM']], 
          result6.loc[:,['','','AVGEARNNORM']],
          result7.loc[:,['','','MAXBURNNORM']],
          result8.loc[:,['','','AVGBURNNORM']]
         ]

In [ ]:
# set index to account number and month
r12 = frames[0].set_index(['', '']).join(frames[1].set_index(['', '']))

In [ ]:
r12

In [ ]:
# set index to account number and month
r123 = r12.join(frames[2].set_index(['', '']))

In [ ]:
r123

In [ ]:
# set index to account number and month
r1234 = r123.join(frames[3].set_index(['', '']))

In [ ]:
r1234

In [ ]:
# set index to account number and month
r12345 = r1234.join(frames[4].set_index(['', '']))

In [ ]:
r12345

In [ ]:
# set index to account number and month
r123456 = r12345.join(frames[5].set_index(['', '']))

In [ ]:
r123456

In [ ]:
# set index to account number and month
r1234567 = r123456.join(frames[6].set_index(['', '']))

In [ ]:
r1234567 = r1234567.replace(np.nan, -1)

In [ ]:
r1234567

In [ ]:
# set index to account number and month
r12345678 = r1234567.join(frames[7].set_index(['', '']))

In [ ]:
r12345678 = r12345678.replace(np.nan, -1)

In [ ]:
r12345678.describe()

In [ ]:
r12345678 = r12345678.reset_index()
r12345678 = r12345678[r12345678[''] != ''] # take out a certain account number 
# set index to account number and month
r12345678 = r12345678.set_index(['',''])

In [ ]:
save = r12345678.reset_index()

In [ ]:
save

In [ ]:
# check for duplicate in account number and month
burn_error = save.reset_index().duplicated(subset=['',''], keep='first')

In [ ]:
burn_error.index[burn_error]

## ranking by z score

In [ ]:
r12345678[(r12345678['SPENDZ'] > 4) &
          (r12345678['AVGSPENDZ'] > 4) &
          (r12345678['TRANSNUMZ'] > 4) &
          (r12345678['MAXSPENDZ'] > 4)&
          (r12345678['MAXEARNZ'] > 4) &
          (r12345678['AVGEARNZ'] > 4) &
          (r12345678['MAXBURNZ'] > 4) &
          (r12345678['AVGBURNZ'] > 4)].sort_values(
    ['SPENDZ','AVGSPENDZ','TRANSNUMZ','MAXSPENDZ','MAXEARNZ','AVGEARNZ','MAXBURNZ','AVGBURNZ'], ascending=False)

In [ ]:
rank = r12345678.sort_values(
    ['SPENDZ','AVGSPENDZ','TRANSNUMZ','MAXSPENDZ','MAXEARNZ','AVGEARNZ','MAXBURNZ','AVGBURNZ'], ascending=False)

In [ ]:
rank[0:20]

## export to xlsx

In [ ]:
writer = pd.ExcelWriter('dbscan.xlsx')
save.to_excel(writer,'Sheet1',index=False)
writer.save()

# Filter out top spender of 2017

In [ ]:
# group by account numbe in 2017 only
# aggregate sum over the sales amount
top_spend = df[df[''] == 2017].groupby([''])[''].sum().reset_index(name='TOTALSPEND').sort_values(['TOTALSPEND'],ascending=False)

In [ ]:
# set index to account number
top_spend.set_index('')

In [ ]:
# set index to account number
top_spend = top_spend[top_spend.TOTALSPEND > np.percentile(top_spend.TOTALSPEND, 99.9)].set_index('')

In [ ]:
filt = top_spend.reset_index()

In [ ]:
filt

In [ ]:
save

In [ ]:
# check if accounts in 2017 exits in 2018 
save = save[~save.ACCOUNTNUM.isin(list(filt['']))]